In [1]:
import torch
from torch.nn.utils import prune

from tqdm import tqdm

from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset

from utils.prune_utils import sparsegpt_prune

c:\Users\phill\anaconda3\envs\pyt-exp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#DEVICE
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_size = "opt-125m"

model_name = f"facebook/{model_size}"

#Load dataset
dataset = load_dataset('wikitext', "wikitext-2-raw-v1", streaming=True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

In [3]:
# Calibrate model (get inputs to each layer with calibration data)

calibration_size=4
token_length=1024
calibration_batch_size=2

EPSILON = 0.01
B = 4
Bs = 2

# run model on batches of calibration data, then concatenate inputs
def split_model_calibration(model):
    batch_sentences = []
    for i, data in tqdm(enumerate(iter(dataset['train'])), total=calibration_size):
        if i < calibration_size + 1:
            if len(batch_sentences) >= calibration_batch_size:
                with torch.no_grad():
                    encoded_input = tokenizer(batch_sentences, return_tensors="pt",
                                              padding="max_length", max_length=token_length,
                                              truncation=True).to(device=device)
                    model(**encoded_input, labels=encoded_input.input_ids)
                    batch_sentences = []
            batch_sentences.append(data['text'])
        else:
            break

# Sparsify Model

In [4]:
# from iterative_prune_finetune import iterative_sparsegpt_prune_tune
from utils.prehook_utils import put_input_hooks,remove_all_hooks
from utils.prune_utils import sparsegpt_prune
from utils.finetune_utils import finetune_model_inplace
from utils.save_utils import unmask_model
from utils.save_utils import mask_from_pruned, unmask_model
import gc

def get_prop_zeros(model):
    return torch.sum(model.get_decoder().layers[0].self_attn.k_proj.weight == 0) / torch.numel(model.get_decoder().layers[0].self_attn.k_proj.weight)

SPARSITIES = [0.9, 0.7, 0.5, 0.3, 0.2]#0.1, 0.2,0.3,0.5,0.7,0.9,1
for SPARSENESS in SPARSITIES:
    model = OPTForCausalLM.from_pretrained(f'facebook/{model_size}', 
                                       output_attentions=True, 
                                       output_hidden_states=True).to(device=device)
    # model = torch.nn.DataParallel(model, device_ids=[0,1,2,3])

    feature_hessians = {}
    #put_input_hooks(model=model, features=feature_hessians, storage_dir=storage_dir, offload_freq=10000, feature_storage_device='cpu')
    all_hooks = put_input_hooks(model=model, features=feature_hessians, feature_storage_device='cpu')
    split_model_calibration(model)
    for hook in all_hooks:
        hook.remove()
    print(f"memory allocated before pruning: {torch.cuda.memory_allocated()}")
    sparsegpt_prune(model=model, model_name=model_size, 
                    feature_hessians=feature_hessians, 
                    EPSILON=EPSILON, SPARSENESS=SPARSENESS, B=B, Bs=Bs, save_model=False)
    feature_hessians.clear()
    gc.collect()
    torch.cuda.empty_cache()
    print(f"memory allocated before fine tuning: {torch.cuda.memory_allocated()}")
    mask_from_pruned(model=model)
    finetune_model_inplace(model=model, tokenizer=tokenizer, 
                           SPARSITY=SPARSENESS, device=device, EPOCH_COUNT=10)
    unmask_model(model=model)
    torch.cuda.empty_cache()
    #unmask_model(model=model)
    pruned_model_name = f'{model_size}-finetuned-{SPARSENESS}'
    torch.save(model.state_dict(), f'pruned_models/{pruned_model_name}-iterative.pt')
    print(f"memory allocated: {torch.cuda.memory_allocated()}")
    print(f"proportion zeros: {get_prop_zeros(model)}")
# Prune using the sparseGPT method, saves as pruned_models/{model_name}-{SPARSENESS}.pt WITHOUT mask
#iterative_sparsegpt_prune_tune(model, model_size, SPARSITIES, feature_hessians, EPSILON, B, Bs, tokenizer, EPOCH_COUNT=10)


5it [00:05,  1.09s/it]                       


memory allocated before pruning: 502228992


100%|██████████| 196/196 [00:50<00:00,  3.85it/s]


memory allocated before fine tuning: 502228992


  0%|          | 0/10 [00:00<?, ?it/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


...

memory allocated after epoch: 2728095232


memory allocated: 1005854720
proportion zeros: 0.30078125


5it [00:07,  1.44s/it]                       


memory allocated before pruning: 503236608


 38%|███▊      | 75/196 [00:18<00:27,  4.33it/s]

In [6]:
for k in feature_hessians.keys():
    print(k)

model.decoder.project_in
model.decoder.layers.0.self_attn.in_proj
model.decoder.layers.0.self_attn.out_proj
model.decoder.layers.0.fc1
model.decoder.layers.0.fc2
model.decoder.layers.1.self_attn.in_proj
model.decoder.layers.1.self_attn.out_proj
model.decoder.layers.1.fc1
model.decoder.layers.1.fc2
model.decoder.layers.2.self_attn.in_proj
model.decoder.layers.2.self_attn.out_proj
model.decoder.layers.2.fc1
model.decoder.layers.2.fc2
model.decoder.layers.3.self_attn.in_proj
model.decoder.layers.3.self_attn.out_proj
model.decoder.layers.3.fc1
model.decoder.layers.3.fc2
model.decoder.layers.4.self_attn.in_proj
model.decoder.layers.4.self_attn.out_proj
model.decoder.layers.4.fc1
model.decoder.layers.4.fc2
model.decoder.layers.5.self_attn.in_proj
model.decoder.layers.5.self_attn.out_proj
model.decoder.layers.5.fc1
model.decoder.layers.5.fc2
model.decoder.layers.6.self_attn.in_proj
model.decoder.layers.6.self_attn.out_proj
model.decoder.layers.6.fc1
model.decoder.layers.6.fc2
model.decoder.la